# Datawhale 零基础入门数据挖掘-Baseline

## Baseline-v1.0 版

Tip:这是一个最初始baseline版本,抛砖引玉,为大家提供一个基本Baseline和一个竞赛流程的基本介绍，欢迎大家多多交流。

**赛题：零基础入门数据挖掘 - 二手车交易价格预测**

地址：https://tianchi.aliyun.com/competition/entrance/231784/introduction?spm=5176.12281957.1004.1.38b02448ausjSX

### Step 1:导入函数工具箱

In [1]:
## 基础工具
import numpy as np
import pandas as pd
import warnings
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import jn
from IPython.display import display, clear_output
import time
from tqdm import tqdm
from sklearn.preprocessing import PolynomialFeatures

warnings.filterwarnings('ignore')
%matplotlib inline

## 模型预测的
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from xgboost.sklearn import XGBRegressor
from lightgbm.sklearn import LGBMRegressor

## 数据降维处理的
from sklearn.decomposition import PCA,FastICA,FactorAnalysis,SparsePCA

import lightgbm as lgb
import xgboost as xgb

## 参数搜索和评价的
from sklearn.model_selection import GridSearchCV,cross_val_score,StratifiedKFold,train_test_split,KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer

### Step 2:数据读取

In [2]:
# 优化类型，减少内存消耗
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() 
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() 
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
## 通过Pandas对于数据进行读取 (pandas是一个很友好的数据读取函数库)
Train_data = pd.read_csv('./data/used_car_train_20200313.csv', sep=' ')
TestA_data = pd.read_csv('./data/used_car_testA_20200313.csv', sep=' ')

## 输出数据的大小信息
print('Train data shape:',Train_data.shape)
print('TestA data shape:',TestA_data.shape)

Train data shape: (150000, 31)
TestA data shape: (50000, 30)


In [4]:
#reduce_mem_usage(Train_data)
#reduce_mem_usage(TestA_data)

In [5]:
## 通过 .columns 查看列名
Train_data.columns

Index(['SaleID', 'name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType',
       'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode',
       'seller', 'offerType', 'creatDate', 'price', 'v_0', 'v_1', 'v_2', 'v_3',
       'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12',
       'v_13', 'v_14'],
      dtype='object')

### Step 3:特征与标签构建

### Step 4: 特征工程

In [6]:
Train_data = Train_data.drop(Train_data[Train_data['v_14'] > 8].index)
Train_data = Train_data.drop(Train_data[Train_data['price'] < 3].index)

In [7]:
def Feature_engineer(df):
    '''1. 处理power'''
    # 截断，截断处为2500。取log并归一化
    df.loc[df['power'] > 2500,'power'] = 2500
    min_max_scaler = preprocessing.MinMaxScaler()
    df['power'] = np.log(df['power'] + 1) 
    df['power'
      ] = ((df['power'] - np.min(df['power'])) / (np.max(df['power']) - np.min(df['power'])))
    
    '''2. 处理 createdate和regdate'''
    ## 分别拆分 年 月 日 然后createDate-regDate得到使用时间
    date_cols = ['regDate', 'creatDate']
    def date_proc(x):
        m = int(x[4:6])
        if m == 0:
            #m = 1
            m = x[6:]
        return x[:4] + '-' + str(m) + '-' + x[6:]
    for f in tqdm(date_cols):
        df[f] = pd.to_datetime(df[f].astype('str').apply(date_proc))
        df[f + '_year'] = df[f].dt.year
        df[f + '_month'] = df[f].dt.month
        df[f + '_day'] = df[f].dt.day
        df[f + '_dayofweek'] = df[f].dt.dayofweek
    df['used_time_day'] = (pd.to_datetime(df['creatDate'], format='%Y%m%d', errors='coerce') - 
                                pd.to_datetime(df['regDate'], format='%Y%m%d', errors='coerce')).dt.days
    df['used_time_month'] = round(df['used_time_day'] / 30)
    df['used_time_year'] = round(df['used_time_day'] / 365)
    df['creatDate'] = df['creatDate'].dt.year * 10000 + df['creatDate'].dt.month * 100 + df['creatDate'].dt.day
    df['regDate'] = df['regDate'].dt.year * 10000 + df['regDate'].dt.month * 100 + df['regDate'].dt.day
    
    df['year_kilometer'] = df['kilometer'] / df['used_time_year']
    
    '''3. 处理fueltype'''
    # 合并2-6为一类
    df.loc[df['fuelType'] == 3,'fuelType'] = 2 
    df.loc[df['fuelType'] == 4,'fuelType'] = 2 
    df.loc[df['fuelType'] == 5,'fuelType'] = 2 
    df.loc[df['fuelType'] == 6,'fuelType'] = 2 
    
    '''4. 处理seller，offerType，直接删除'''
    df = df.drop(['seller'], axis=1)
    df = df.drop(['offerType'], axis=1)
    
    
    '''5. 处理notRepairedDamage'''
    df['notRepairedDamage'].replace('-', np.nan , inplace=True)
    df['notRepairedDamage'] = df['notRepairedDamage'].astype('float64')

    '''6. 基于业务知识构建特征'''
    # 年限和行驶里程折旧
    def depreciation_year1(year):
        if year <= 3:
            return 1 - year * 0.15
        elif year > 3 and  year <= 7:
            return 0.55 - (year-3) * 0.1
        elif year > 7 and  year <= 10:
            return 0.25 - (year-7) * 0.05
        else:
            return 0
    df['depreciation_year1'] = df['used_time_year'].apply(lambda x: depreciation_year1(x))

    def depreciation_year2(year):
        if year <= 3:
            return 1 - 0.85 * year * 0.11
        elif year > 3 and  year <= 7:
            return 0.7195 - 0.85 * (year-3) * 0.1
        elif year > 7 and  year <= 10:
            return 0.3795 - 0.85 * (year-7) * 0.09
        else:
            return 0.15

    df['depreciation_year2'] = df['used_time_year'].apply(lambda x: depreciation_year2(x))    
    
    def depreciation_kilometer(kilo):
        if kilo <= 6:
            return 1 - kilo * 5 / 90
        elif kilo > 6 and  kilo <= 12:
            return 0.66667 - (kilo-6) * 4 / 90
        elif kilo > 12 and  kilo <= 18:
            return 0.4 - (kilo-12) * 3 / 90
        elif kilo > 18 and  kilo <= 24:
            return 0.2 - (kilo-18) * 2 / 90
        elif kilo > 24 and  kilo <= 30:
            return 0.06667 - (kilo-24) * 1 / 90
    df['depreciation_kilo'] = df['used_time_year'].apply(lambda x: depreciation_year1(x))
    
    '''7. 构建统计量'''
    
    # 分类变量的统计
    def feat_uniqiue(df,feature):
        Train_gb = df.groupby(feature)
        all_info = {}
        for kind, kind_data in Train_gb:
            info = {}
            info[feature + '_unique'] = len(kind_data[feature].unique())
            info[feature + '_amount'] = len(kind_data)
            all_info[kind] = info
        brand_fe = pd.DataFrame(all_info).T.reset_index().rename(columns={'index': feature})
        return df.merge(brand_fe, how='left', on=feature)    
    
    category_feature = ['name', 'model', 'regionCode','notRepairedDamage','brand','gearbox','fuelType','bodyType']
    for i in category_feature:
        df = feat_uniqiue(df,i) 
    
    # 两个变量间交互构建 max median min sum std average等变量
    def Group_Statistic(df,feature,Target):
        Train_gb = df.groupby(feature)
        all_info = {}
        for kind, kind_data in Train_gb:
            info = {}
            kind_data = kind_data[kind_data[Target] > 0]
            #info[feature + '_unique'] = len(kind_data[feature].unique())
            #info[feature + '_amount'] = len(kind_data)
            info[feature + '_' + Target + '_max'] = kind_data[Target].max()
            info[feature + '_' + Target + '_median'] = kind_data[Target].median()
            info[feature + '_' + Target + '_min'] = kind_data[Target].min()
            info[feature + '_' + Target + '_sum'] = kind_data[Target].sum()
            info[feature + '_' + Target + '_std'] = kind_data[Target].std()
            info[feature + '_' + Target + '_average'] = round(kind_data[Target].sum() / (len(kind_data) + 1), 2)
            all_info[kind] = info
        brand_fe = pd.DataFrame(all_info).T.reset_index().rename(columns={'index': feature})
        return df.merge(brand_fe, how='left', on=feature)
    
    # 和price有关的无法直接应用在test上，需要从train上挪过去。
    df['model'] = df['model'].map(lambda x:str(x))
    df['brand'] =df['brand'].map(lambda x:str(x))
    df['Model_brand'] = df['model'].str.cat(df['brand'], sep = '_')
    #df = Group_Statistic(df,'brand','price')
    #df = Group_Statistic(df,'model','price')
    #df = Group_Statistic(df,'kilometer','price')
    #df = Group_Statistic(df,'regionCode','price')
    #df = Group_Statistic(df,'used_time_year','price')
    #df = Group_Statistic(df,'used_time_month','price')
    #df = Group_Statistic(df,'used_time_day','price')
    #df = Group_Statistic(df,'power','price')
    #df = Group_Statistic(df,'fuelType','price')
    #df = Group_Statistic(df,'notRepairedDamage','price')
   # df = Group_Statistic(df,'bodyType','price')

    df = Group_Statistic(df,'regionCode','used_time_day')
    df = Group_Statistic(df,'regionCode','used_time_month')
    df = Group_Statistic(df,'regionCode','used_time_year')
    df = Group_Statistic(df,'regionCode','kilometer')
    df = Group_Statistic(df,'regionCode','power')
    df = Group_Statistic(df,'regionCode','v_3')
    
    df = Group_Statistic(df,'used_time_year','v_3')
    df = Group_Statistic(df,'used_time_month','v_3')
    df = Group_Statistic(df,'used_time_day','v_3')
    
    df = Group_Statistic(df,'power','used_time_day')
    df = Group_Statistic(df,'power','used_time_month')
    df = Group_Statistic(df,'power','used_time_year')
    
    df = Group_Statistic(df,'brand','used_time_day')
    df = Group_Statistic(df,'brand','used_time_month')
    df = Group_Statistic(df,'brand','used_time_year')
    df = Group_Statistic(df,'model','used_time_day')
    df = Group_Statistic(df,'model','used_time_month')
    df = Group_Statistic(df,'model','used_time_year')
    df = Group_Statistic(df,'brand','kilometer')
    df = Group_Statistic(df,'brand','power')
    df = Group_Statistic(df,'model','kilometer')
    df = Group_Statistic(df,'model','power')
    
    #df = Group_Statistic(df,'Model_brand','price')
    df = Group_Statistic(df,'Model_brand','used_time_day')
    df = Group_Statistic(df,'Model_brand','used_time_month')
    df = Group_Statistic(df,'Model_brand','used_time_year')
    df = Group_Statistic(df,'Model_brand','power')
    df = Group_Statistic(df,'Model_brand','kilometer')
    # df = df.drop(['Model_brand'], axis=1)
    df['Model_brand'] = df['Model_brand'].astype('float64')
    df['model'] = df['model'].astype('float64')
    df['brand'] = df['brand'].astype('float64')

    return df
   

In [8]:
# 合并数据
Train_data.index = Train_data['SaleID']
TestA_data.index = TestA_data['SaleID']
Data_all = pd.concat([Train_data, TestA_data], sort=False).reset_index(drop=True)
Data_all.index = Data_all['SaleID']
reduce_mem_usage(Data_all)

Memory usage of dataframe is 51199744.00 MB
Memory usage after optimization is: 15200028.00 MB
Decreased by 70.3%


,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
SaleID,,,,,,,,,,,,,,,,,,,,,
0,0,736,20040402,30.0,6,1.0,0.0,0.0,60,12.5,...,0.235718,0.101990,0.129517,0.022812,0.097473,-2.880859,2.804688,-2.419922,0.795410,0.914551
1,1,2262,20030301,40.0,1,2.0,0.0,0.0,0,15.0,...,0.264893,0.121033,0.135742,0.026596,0.020584,-4.902344,2.095703,-1.030273,-1.722656,0.245483
2,2,14874,20040403,115.0,15,1.0,0.0,0.0,163,12.5,...,0.251465,0.114929,0.165161,0.062164,0.027069,-4.847656,1.803711,1.565430,-0.832520,-0.229980
3,3,71865,19960908,109.0,10,0.0,0.0,1.0,193,15.0,...,0.274414,0.110291,0.121948,0.033386,0.000000,-4.507812,1.286133,-0.501953,-2.437500,-0.478760
4,4,111080,20120103,110.0,5,1.0,0.0,0.0,68,5.0,...,0.228027,0.073181,0.091858,0.078796,0.121521,-1.896484,0.910645,0.931152,2.833984,1.923828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199995,20903,19960503,4.0,4,4.0,0.0,0.0,116,15.0,...,0.284668,0.130005,0.049835,0.028809,0.004616,-5.976562,1.302734,-1.207031,-1.981445,-0.357666
199996,199996,708,19991011,0.0,0,0.0,0.0,0.0,75,15.0,...,0.268066,0.108093,0.066040,0.025467,0.025970,-3.914062,1.759766,-2.076172,-1.155273,0.169067
199997,199997,6693,20040412,49.0,1,0.0,1.0,1.0,224,15.0,...,0.269531,0.105713,0.117676,0.057465,0.015671,-4.640625,0.654785,1.137695,-1.390625,0.254395


In [9]:
Data_all = Feature_engineer(Data_all)
Data_all = Data_all.drop(['Model_brand'], axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.81it/s]


In [10]:
Data_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199999 entries, 0 to 199998
Columns: 222 entries, SaleID to Model_brand_kilometer_average
dtypes: float16(19), float32(1), float64(180), int16(1), int32(2), int64(19)
memory usage: 315.1 MB


In [11]:
 '''拆分数据集，并对price 做log处理'''
feature_cols = [col for col in Data_all.columns if col not in ['SaleID','price']]

X_data = Data_all[feature_cols][Data_all['SaleID'] <= 149999]
Y_data = Data_all['price'][Data_all['SaleID'] <= 149999]
Y_data = np.log(Y_data + 1)

X_test = Data_all[feature_cols][Data_all['SaleID'] > 149999]

### Step 5:模型训练与预测

#### 1) 利用xgb进行五折交叉验证查看模型的参数效果

In [ ]:
## xgb-Model

xgr = xgb.XGBRegressor(n_estimators=150, learning_rate=0.1, gamma=0, subsample=0.8,\
        colsample_bytree=0.9, max_depth=7) #,objective ='reg:squarederror'

scores_train = []
scores = []

## 5折交叉验证方式
sk=KFold(n_splits=5,shuffle=True,random_state=0)
for train_ind,val_ind in sk.split(X_data,Y_data):
    
    train_x=X_data.iloc[train_ind].values
    train_y=Y_data.iloc[train_ind]
    val_x=X_data.iloc[val_ind].values
    val_y=Y_data.iloc[val_ind]
    
    xgr.fit(train_x,train_y)
    pred_train_xgb=xgr.predict(train_x)
    pred_xgb=xgr.predict(val_x)
    
    score_train = mean_absolute_error(train_y,pred_train_xgb)
    scores_train.append(score_train)
    score = mean_absolute_error(val_y,pred_xgb)
    scores.append(score)

print('Train mae:',np.mean(score_train))
print('Val mae',np.mean(scores))

[17:06:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
train_y_Origin = np.exp(train_y) - 1
pred_train_xgb_Origin = np.exp(pred_train_xgb) - 1
Train_Score_MAE = mean_absolute_error(train_y_Origin,pred_train_xgb_Origin)
print(Train_Score_MAE)

In [ ]:
val_y_Origin = np.exp(val_y) - 1
pred_xgb_Origin = np.exp(pred_xgb) - 1
val_Score_MAE = mean_absolute_error(val_y_Origin,pred_xgb_Origin)
print(val_Score_MAE)

In [ ]:
# 特征重要度排名
feature_cols = X_data.columns
feat_imp_df = pd.DataFrame({'feat': feature_cols, 'imp': 0})
feat_imp_df['imp'] = xgr.feature_importances_

plt.figure(figsize=(20, 100))
feat_imp_df = feat_imp_df.sort_values('imp').reset_index(drop=False)
sns.barplot(x='imp', y='feat', data=feat_imp_df)

In [ ]:
## 定义了一个统计函数，方便后续信息统计
def Sta_inf(data):
    print('_min',np.min(data))
    print('_max:',np.max(data))
    print('_mean',np.mean(data))
    print('_ptp',np.ptp(data))
    print('_std',np.std(data))
    print('_var',np.var(data))

In [ ]:
# 建模
x_train,x_val,y_train,y_val = train_test_split(X_data,Y_data,test_size=0.3)

def build_model_xgb(x_train,y_train):
    model = xgb.XGBRegressor(n_estimators=150, learning_rate=0.1, gamma=0, subsample=0.8,\
        colsample_bytree=0.9, max_depth=7) #, objective ='reg:squarederror'
    model.fit(x_train, y_train)
    return model

print('Train xgb...')
model_xgb = build_model_xgb(x_train,y_train)
val_xgb = model_xgb.predict(x_val)
MAE_xgb = mean_absolute_error(y_val,val_xgb)
print('MAE of val with xgb:',MAE_xgb)

In [ ]:
# 预测
print('Predict xgb...')
model_xgb_pre = build_model_xgb(X_data,Y_data)
subA_xgb = model_xgb_pre.predict(X_test)
subA_xgb = np.exp(subA_xgb) - 1
print('Sta of Predict xgb:')
Sta_inf(subA_xgb)

In [ ]:
# 输出结果
sub = pd.DataFrame()
sub['SaleID'] = TestA_data.SaleID
sub['price'] = subA_xgb
sub.to_csv('./sub_Weighted.csv',index=False)